In [1]:
from obsutils.utils import SkyportalClient, get_target_list, get_run_info
from obsutils.data import standards_df
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astroplan import Observer
from astroplan import FixedTarget
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astroplan.plots import plot_airmass
from pathlib import Path
from dotenv import load_dotenv
import os

In [8]:
run = 1888

In [9]:
run_info = get_run_info(run)

In [10]:
night = run_info["calendar_date"].replace("-", "")
night

'20250518'

In [11]:
df = get_target_list(run)
df.sort_values(by=["priority"], ascending=False)[["id", "tns_name", "priority", "comment"]]

,id,tns_name,priority,comment
0,ZTF25aagevje,TDE 2025chm,4,TDE follow-up
2,ZTF25aaooewz,None,4,Good TDE Candidate
5,ZTF25aapwprt,None,4,TDE candidate
4,ZTF24abszkov,AT 2024abmx,3,Worth a good host spectrum?
6,ZTF25aaospja,AT 2025iyw,3,OK tde candidate
7,ZTF18aaotzun,None,3,Probably SN?
9,ZTF20aappkya,None,3,Not bad
1,ZTF25aapadmg,AT 2025kkd,2,OK candidate
3,ZTF25aachmub,None,2,"Filler, AGN probably"
8,ZTF25aaoxgxs,AT 2025ivx,2,Low likelihood TDE


In [12]:
targets = []
tcslist = ['#title=true ra=true dec=true epoch=false muRA=false muDec=false ' +
           'magnitude=false dRA=false dDec=false rotatorPA=true ' +
           'rotatorFrame=true comment=false']

# for i in obslist.index:
for _, row in df.iterrows():
    coords = SkyCoord(ra=row["ra"], dec=row["dec"],
                      unit=(u.deg, u.deg))
    targ = FixedTarget(coord=coords, name=row["id"])
    targets.append(targ)
    fmt_coord = coords.to_string('hmsdms', sep=':', pad=True, precision=2)
    tcslist.append(row["id"] + ' ' + fmt_coord + ' 180 Fixed')

for i, row in standards_df.iterrows():
    name = row["name"]
    coords = SkyCoord(ra=row["ra"], dec=row["dec"], unit=(u.hourangle, u.deg))
    targ = FixedTarget(coord=coords, name=row["name"])
    targets.append(targ)
    fmt_coord = coords.to_string('hmsdms', sep=':', pad=True, precision=2)
    tcslist.append(name + ' ' + fmt_coord + ' 180 Fixed')

tcslist

['#title=true ra=true dec=true epoch=false muRA=false muDec=false magnitude=false dRA=false dDec=false rotatorPA=true rotatorFrame=true comment=false',
 'ZTF25aagevje 07:40:59.04 +16:31:53.62 180 Fixed',
 'ZTF25aapadmg 09:07:26.91 +24:45:22.15 180 Fixed',
 'ZTF25aaooewz 11:40:41.76 +18:14:18.79 180 Fixed',
 'ZTF25aachmub 12:27:03.90 +69:29:28.25 180 Fixed',
 'ZTF24abszkov 13:21:17.65 +33:29:35.62 180 Fixed',
 'ZTF25aapwprt 13:51:23.90 +12:29:31.02 180 Fixed',
 'ZTF25aaospja 15:00:06.60 +27:10:50.41 180 Fixed',
 'ZTF18aaotzun 15:08:33.72 +29:17:33.05 180 Fixed',
 'ZTF25aaoxgxs 15:49:57.25 +23:43:51.21 180 Fixed',
 'ZTF20aappkya 17:45:44.91 +67:50:28.20 180 Fixed',
 'ZTF25aaoxrkb 18:19:08.24 +21:47:10.59 180 Fixed',
 'ZTF19abtonmn 18:48:26.99 +64:20:43.46 180 Fixed',
 'G191-B2B 05:05:30.60 +52:49:56.00 180 Fixed',
 'Feige34 10:39:36.71 +43:06:10.10 180 Fixed',
 'Feige67 12:41:51.83 +17:31:20.50 180 Fixed',
 'Feige110 23:19:58.39 -05:09:55.80 180 Fixed',
 'BD+28d4211 21:51:11.07 +28:51:51

In [16]:
load_dotenv()
base_output_dir = Path(os.getenv("OUTPUT_DIR", str(Path.home()) + "/obsutils"))

output_dir = base_output_dir / night
output_dir.mkdir(parents=True, exist_ok=True)


output_path = output_dir / (night + '.tls')

print(f"Saving to {output_path}")
np.savetxt(output_path, tcslist, fmt='%s')

Saving to /Users/robertstein/Data/observing/20250518/20250518.tls


In [ ]:
# # Set observer at DCT
# dct = Observer.at_site('Lowell')

# ############ CHANGE THESE LINES #############
# #############################################
# start = Time('2025-02-28 01:22', scale='utc')
# end = Time('2025-02-28 07:37', scale='utc')
# #############################################

# delta_t = end-start
# obstime = start + delta_t*np.linspace(0, 1, 75)

# # Colors used to tell targets apart - may need to add more
# color_idx = np.array(['r', 'b', 'g', 'teal', 'orange', 'purple', 'k',
#                       'lime', 'cornflowerblue', 'dimgrey', 'cyan',
#                       'blueviolet', 'deeppink', 'y', 'olive', 'yellow',
#                       'deepskyblue', 'darksalmon', 'yellowgreen', 'aliceblue',
#                       'steelblue', 'firebrick', 'violet', 'maroon',
#                       'darkkhaki', 'mediumslateblue', 'plum', 'crimson',
#                       'sandybrown', 'antiquewhite', 'indigo'])

# # targets = np.array(targets)
# # targets = np.delete(targets, [17, 21, 20, 19, 18])

# plt.figure(figsize=(10, 5))
# # Plot airmass
# for i in range(len(targets)):
#     shade = False
#     if i == 0:  # Used to avoid issues with doing multiple shades?
#         shade = True
#     bonk = plot_airmass(targets[i], dct, obstime, brightness_shading=shade,
#                  style_kwargs=dict(color=color_idx[i]),
#                  use_local_tz=True, min_airmass=1.0, max_airmass=2.0)

# ticks = bonk.get_xticklabels()
# newlabels = []
# for t in ticks:
#     time = int(t.get_text()[:2])
#     if time > 7:
#         time -= 7
#     elif time == 7:
#         time = 12
#     elif time < 7:
#         time = 12 - (7-time)
#     newlabels.append(str(time)+":00")
    
# bonk.set_xticklabels(newlabels)
# plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.5), ncol=3)
# #plt.grid()

# ########### CHANGE THIS LINE ######################
# ###################################################
# plt.xlabel('MST')
# ###################################################

# plt.savefig('./'+obsdate+'/LDT-airmass.pdf', dpi=400, bbox_inches='tight')
# plt.show()

In [ ]:
# # Plot sky
# for i in range(len(targets)):
#     plot_sky(targets[i], dct, obstime, north_to_east_ccw=False,
#              style_kwargs=dict(color=color_idx[i]))

# plt.legend(loc=(1.14, 0))
# # plt.savefig('DCT-sky.png', dpi=300, bbox_inches='tight')
# plt.show()

# # Plot altitude
# for i in range(len(targets)):
#     shade = False
#     if i == 0:
#         shade = True
#     plot_altitude(targets[i], dct, obstime, brightness_shading=shade,
#                   style_kwargs=dict(color=color_idx[i]))

# plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.8), ncol=3)

# ########### CHANGE THIS LINE ######################
# ###################################################
# plt.xlabel('Time from 2019-08-18 [UTC] (MST+7:00)')
# ###################################################

# # plt.savefig('DCT-alt.png', dpi=300, bbox_inches='tight')
# plt.show()